#Load data

In [5]:
from google.colab import files
uploaded = files.upload()

Saving dialogues_018.json to dialogues_018.json


In [6]:
# intents
!mkdir -p dialogue_018/NONE
!mkdir -p dialogue_018/SearchOnewayFlight
!mkdir -p dialogue_018/ReserveOnewayFlight
!mkdir -p dialogue_018/SearchRoundtripFlights
!mkdir -p dialogue_018/ReserveRoundtripFlights

Script para coletar apenas as utterances (frases) do usuário e armazená-las num diretório que representa a intenção(que será nossa classe para a tarefa de classificação de intenção).</br>
<b>atenção</b>: O sistema foi excluído da coleta de utterances porque a resposta do sistema não possui intenção

In [31]:
import json

# text_for_entity_extraction é usado pelo extrator de intenções na parte 2 deste notebook
text_for_entity_extraction = ""
if __name__ == '__main__':
    with open("dialogues_018.json", 'r') as file:
        data_018 = json.load(file)

    for i in range(len(data_018)):
        frames_amount = len(data_018[i]['turns'])
        for j in range(frames_amount):
            current_frame = data_018[i]['turns'][j]
            speaker = current_frame['speaker']
            if speaker == 'USER':
                intent = current_frame['frames'][0]['state']['active_intent']
                utterance = current_frame['utterance']

                file_path = "dialogue_018/" + intent + "/" + str(i) + "_" + str(j) + ".txt"

                f = open(file_path, "w+")
                f.write(utterance)
                f.close()

                text_for_entity_extraction += utterance + " "


Armazenar numa estrutura para treinamento no Keras

In [8]:
import tensorflow as tf

seed = 42

raw_train_ds= tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    validation_split=0.2,
    subset='training',
    seed=seed,
    shuffle=False)

raw_validation_ds = tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    seed=seed,
    shuffle=False)

Found 1052 files belonging to 5 classes.
Using 842 files for training.
Found 1052 files belonging to 5 classes.
Using 210 files for validation.


In [9]:
raw_train_ds.class_names

['NONE',
 'ReserveOnewayFlight',
 'ReserveRoundtripFlights',
 'SearchOnewayFlight',
 'SearchRoundtripFlights']

In [ ]:
for text_batch, label_batch in raw_train_ds:
  print("Utterance: ", text_batch)
  print("Label: ", label_batch)

In [ ]:
print(raw_train_ds)

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


Acho que o Tensor acima é dividido em batches e acredito que isso adiciona uma complexidade na hora de tokenizar, pois teríamos que fazer para cada um dos tensores. Testei resolver fazer uma concatenação de todos os tensores abaixo:

In [11]:
all_text_tensor = tf.concat([text_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_text_tensor

In [12]:
all_labels = tf.concat([label_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_labels

In [13]:
utterances_validation_tensor = tf.concat([text_batch for text_batch, label_batch in raw_validation_ds], axis=0)

In [14]:
validation_labels = tf.concat([label_batch for text_batch, label_batch in raw_validation_ds], axis=0)

# Pre processamento de dados
Fazer uma bag-of-words

In [15]:
from keras.preprocessing.text import Tokenizer
import json
import numpy
from keras.preprocessing import text, sequence

### Train subset

In [ ]:
t = Tokenizer()
texts = all_text_tensor.numpy().astype(str)
t.fit_on_texts(texts)

# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(texts, mode='count')
print(encoded_docs)

In [17]:
seq_token = t.texts_to_sequences(texts)
padded_sequences = sequence.pad_sequences(seq_token)

In [18]:
padded_sequences.shape

(842, 30)

In [19]:
all_labels.shape

TensorShape([842])

### Validation subset

In [20]:
t2 = Tokenizer()
validation_texts = utterances_validation_tensor.numpy().astype(str)
t2.fit_on_texts(validation_texts)
validation_seq_token = t2.texts_to_sequences(validation_texts)
padded_validation = sequence.pad_sequences(validation_seq_token)
#padded_validation = sequence.pad_sequences(validation_seq_token, maxlen=30)

In [21]:
padded_validation.shape

(210, 28)

In [22]:
validation_labels.shape

TensorShape([210])

# Classificação de intenções (Parte 1)
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM </br>
https://keras.io/examples/nlp/text_classification_with_transformer/ </br>

## LSTM

In [23]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM
from keras.layers import Embedding
import keras.backend as K

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=842, output_dim=100))
model_lstm.add(LSTM(units=64))
model_lstm.add(Dense(5,activation='softmax'))
model_lstm.compile(loss=['sparse_categorical_crossentropy'] , optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         84200     
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 126765 (495.18 KB)
Trainable params: 126765 (495.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
history_lstm = model_lstm.fit(padded_sequences, all_labels, epochs=10)

Epoch 1/10
27/27 [==============================] - 3s 21ms/step - loss: 1.3951 - accuracy: 0.4656
Epoch 2/10
27/27 [==============================] - 1s 30ms/step - loss: 1.1215 - accuracy: 0.6390
Epoch 3/10
27/27 [==============================] - 1s 34ms/step - loss: 0.8584 - accuracy: 0.7114
Epoch 4/10
27/27 [==============================] - 1s 34ms/step - loss: 0.7038 - accuracy: 0.7470
Epoch 5/10
27/27 [==============================] - 1s 25ms/step - loss: 0.5866 - accuracy: 0.7838
Epoch 6/10
27/27 [==============================] - 1s 20ms/step - loss: 0.5102 - accuracy: 0.8361
Epoch 7/10
27/27 [==============================] - 1s 20ms/step - loss: 0.4534 - accuracy: 0.8480
Epoch 8/10
27/27 [==============================] - 1s 21ms/step - loss: 0.3997 - accuracy: 0.8622
Epoch 9/10
27/27 [==============================] - 1s 20ms/step - loss: 0.3630 - accuracy: 0.8872
Epoch 10/10
27/27 [==============================] - 1s 21ms/step - loss: 0.3320 - accuracy: 0.8955


In [25]:
history_with_validation_lstm = model_lstm.fit(padded_sequences, all_labels, validation_data=(padded_validation, validation_labels), epochs=10)

Epoch 1/10
27/27 [==============================] - 2s 57ms/step - loss: 0.3185 - accuracy: 0.8967 - val_loss: 2.6472 - val_accuracy: 0.3143
Epoch 2/10
27/27 [==============================] - 1s 24ms/step - loss: 0.2852 - accuracy: 0.9097 - val_loss: 2.4851 - val_accuracy: 0.4048
Epoch 3/10
27/27 [==============================] - 1s 23ms/step - loss: 0.2805 - accuracy: 0.9038 - val_loss: 3.0960 - val_accuracy: 0.3238
Epoch 4/10
27/27 [==============================] - 1s 24ms/step - loss: 0.2650 - accuracy: 0.9157 - val_loss: 3.4387 - val_accuracy: 0.3095
Epoch 5/10
27/27 [==============================] - 1s 23ms/step - loss: 0.2584 - accuracy: 0.9109 - val_loss: 3.1308 - val_accuracy: 0.3000
Epoch 6/10
27/27 [==============================] - 1s 24ms/step - loss: 0.2184 - accuracy: 0.9252 - val_loss: 2.7952 - val_accuracy: 0.3333
Epoch 7/10
27/27 [==============================] - 1s 36ms/step - loss: 0.2061 - accuracy: 0.9311 - val_loss: 2.9650 - val_accuracy: 0.3714
Epoch 8/10
27

## SVM

In [ ]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

with open("dialogues_018.json", 'r') as file:
    data_018 = json.load(file)

texts = []
labels = []

for i in range(len(data_018)):
    frames_amount = len(data_018[i]['turns'])
    for j in range(frames_amount):
        current_frame = data_018[i]['turns'][j]
        speaker = current_frame['speaker']
        if speaker == 'USER':
            intent = current_frame['frames'][0]['state']['active_intent']
            utterance = current_frame['utterance']
            texts.append(utterance)
            labels.append(intent)

texts_train, texts_validation, labels_train, labels_validation = train_test_split(texts, labels, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(texts_train)
X_validation = vectorizer.transform(texts_validation)

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, labels_train)

predictions = svm_model.predict(X_validation)

accuracy = accuracy_score(labels_validation, predictions)
print(f'Acurácia do modelo SVM: {accuracy}')


Acurácia do modelo SVM: 0.8104265402843602


In [ ]:
# Exemplo de diálogo
example_dialogue = [
    {"speaker": "USER", "utterance": "I would like a short trip to New York"}
    # Adicione mais turnos do diálogo conforme necessário
]

# Pré-processamento do exemplo de diálogo
example_text = example_dialogue[0]['utterance']
example_vector = vectorizer.transform([example_text])

# Previsão do modelo SVM
predicted_intent = svm_model.predict(example_vector)

# Resultados
print(f"Texto do Usuário: {example_text}")
print(f"Intenção Prevista: {predicted_intent[0]}")


Texto do Usuário: I would like a short trip to New York
Intenção Prevista: SearchRoundtripFlights


## CNN

In [ ]:
import pandas as pd
import numpy as np
import time

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Embedding(input_dim=842, output_dim=100))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Alterado para categorical_crossentropy
              metrics=['accuracy'])

print(model.summary())

# Convertendo os rótulos para codificação one-hot
from tensorflow.keras.utils import to_categorical
labels_one_hot = to_categorical(all_labels, num_classes=5)

# Fit do modelo
history = model.fit(padded_sequences, labels_one_hot, epochs=10, validation_data=(padded_validation, to_categorical(validation_labels, num_classes=5)), batch_size=5)


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 100)         84200     
                                                                 
 conv1d_8 (Conv1D)           (None, None, 128)         64128     
                                                                 
 global_max_pooling1d_8 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dense_17 (Dense)            (None, 5)                 325       
                                                                 
Total params: 156909 (612.93 KB)
Trainable params: 156909 (612.93 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [ ]:
# Supondo que você tenha um novo texto a ser classificado
new_text = "i need to buy a one way flight"

new_sequence = t.texts_to_sequences([new_text])
padded_new_sequence = pad_sequences(new_sequence, maxlen=10, padding='post')

predictions = model.predict(padded_new_sequence)

predicted_class = predictions.argmax(axis=-1)

print(f'O texto foi classificado como classe: {predicted_class[0]}')


1/1 [==============================] - 0s 115ms/step
O texto foi classificado como classe: 1


## Teste de transformer com BERT

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.layers import Dense

In [ ]:
# Load BERT tokenizer and model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Modify the model's classifier layer
classifier_layer = Dense(5, activation='softmax',  name='classifier')
bert_model.layers[-1]= classifier_layer

In [ ]:
import keras
opt = keras.optimizers.Adam(learning_rate=2e-5)
bert_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bert_history = bert_model.fit(
    padded_sequences,
    all_labels,
    epochs=3
)

Epoch 1/3
27/27 [==============================] - 328s 11s/step - loss: 3.1591 - accuracy: 0.3753
Epoch 2/3
27/27 [==============================] - 279s 10s/step - loss: 1.9506 - accuracy: 0.3717
Epoch 3/3
27/27 [==============================] - 283s 10s/step - loss: 1.4578 - accuracy: 0.4204


# Extrator de entidades (Parte 2)

In [ ]:
!pip install spacy -q
!python -m spacy download en_core_web_sm

In [28]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp(text_for_entity_extraction)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.is_stop)

In [34]:
import pandas as pd

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)

df

,text,lemma,POS,explain,stopword
0,I,I,PRON,pronoun,True
1,need,need,VERB,verb,False
2,a,a,DET,determiner,True
3,one,one,NUM,numeral,True
4,way,way,NOUN,noun,False
...,...,...,...,...,...
10991,That,that,PRON,pronoun,True
10992,will,will,AUX,auxiliary,True
10993,be,be,AUX,auxiliary,True
10994,all,all,PRON,pronoun,True


In [35]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

# Conversação

In [44]:
def preprocess_input(user_input):
  t = Tokenizer()
  t.fit_on_texts(user_input)
  seq_token = t.texts_to_sequences([user_input])
  padded_sequences = sequence.pad_sequences(seq_token)
  return padded_sequences

In [51]:
import numpy as np
def classify_intent(preprocessed_input):
  predictions = model_lstm.predict(np.array(preprocessed_input))

  predicted_intent_index = np.argmax(predictions)
  confidence = predictions[0][predicted_intent_index]

  intents = ["ReserveOnewayFlight", "SearchOnewayFlight", "SearchRoundtripFlights", "ReserveRoundtripFlights", "NONE"]

  predicted_intent = intents[predicted_intent_index]

  return predicted_intent, confidence

In [46]:
def extract_entities(doc):
  entities = []

  for ent in doc.ents:
      entities.append({
          'text': ent.text,
          'start_char': ent.start_char,
          'end_char': ent.end_char,
          'label': ent.label_
      })

  return entities

In [ ]:
def generate_response(combined_info):
  pass

In [53]:
while True:
  user_input = input("User: ")

  if user_input == "Bye":
    break

  # Preprocess the user input
  preprocessed_input = preprocess_input(user_input)
  #print(preprocessed_input.shape)

  # Intent Classification
  intent, confidence = classify_intent(preprocessed_input)

  # Entity Extraction
  doc = nlp(user_input)
  entities = extract_entities(doc)

  # Combine Information
  combined_info = {'intent': intent, 'confidence': confidence, 'entities': entities}

  # Dialogue Management and Response Generation
  #response = generate_response(combined_info)

  print(f"Intent: {intent}, Confidence: {confidence}")
  print(f"Entities: {entities}")
  #print(f"Bot: {response}\n")

User: Bye
